# Aprendizado Supervisionado II - Trabalho 2

## Pacotes

In [ ]:
import pandas as pd

import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt

from sklearn.datasets import load_iris

from sklearn.manifold import TSNE
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, plot_confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
#from utils import make_dist_plot, make_roc_curve

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

from sklearn import metrics

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Pré-Processamento

In [ ]:
wine = pd.read_csv('/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')

In [ ]:
#Criando categorias "Ruim" e "Bom" para os níveis de qualidade.
#'Ruim' = 0 até 6, 'Bom' = 7 até 10
bins = [0, 7, 10]
names = ['Ruim', 'Bom']

d = dict(enumerate(names, 1))

wine['categoria'] = np.vectorize(d.get)(np.digitize(wine['quality'], bins))

Foram definidas duas categorias para a classificação da qualidade: 'Ruim' = 0 até 6, 'Bom' = 7 até 10. Observamos um total de 1382 observações como 'Ruim' e 217 como 'Bom'. Não houveram observações com valores nulos.

In [ ]:
wine.head()

In [ ]:
wine.info()

In [ ]:
wine["categoria"].value_counts()

In [ ]:
wine.describe()

In [ ]:
wine.isnull().sum()

## Análise Exploratória

Foram observadas 86.4% das observações como 'Ruim' e apenas 13.6% como 'Bom'. O gráfico de barras mostra que a maior parte das observações está concentrada entre notas de qualidade '5' e '6', englobadas dentro da categoria 'Ruim'. Além disso, a menor das observações tem nota em qualidade '3' e a maior tem nota '8'.

In [ ]:
wine['categoria'].value_counts().plot(kind = 'pie', explode = [0, 0.1], figsize = (6, 6), autopct = '%1.1f%%', shadow = True)
plt.ylabel('Bom vs Ruim')
plt.legend(['Ruim', 'Bom'])
plt.show()

In [ ]:
fig, axs = plt.subplots(ncols=2,figsize=(9.7, 6.27))

sns.countplot(x='quality',data=wine, ax=axs[0])
sns.countplot(x="categoria",data=wine, ax=axs[1])

### Boxplots

Os boxplots evidenciam um grande número de outliers para a maioria das variáveis. Podemos observar que para a a categoria 'Bom' as variáveis 'fixed acidity', 'citric acid', 'sulphates' e 'alcohol' tem médias maiores do que para 'Ruim'. Já 'volatile acidity', 'free sulfur dioxide', 'total sulfor dioxide' e 'density' tem médias maiores para 'Ruim'. As variáveis 'residual sugar', 'chlorides' e 'ph' tem médias semelhantes para ambas as categorias.

In [ ]:
fig, axs = plt.subplots(ncols=2,figsize=(9.7, 6.27))

sns.boxplot(x="quality",y='fixed acidity',data=wine, ax=axs[0])
sns.boxplot(x="categoria",y='fixed acidity',data=wine, ax=axs[1])

In [ ]:
fig, axs = plt.subplots(ncols=2,figsize=(9.7, 6.27))

sns.boxplot(x="quality",y='volatile acidity',data=wine, ax=axs[0])
sns.boxplot(x="categoria",y='volatile acidity',data=wine, ax=axs[1])

In [ ]:
fig, axs = plt.subplots(ncols=2,figsize=(9.7, 6.27))

sns.boxplot(x="quality",y='citric acid',data=wine, ax=axs[0])
sns.boxplot(x="categoria",y='citric acid',data=wine, ax=axs[1])

In [ ]:
fig, axs = plt.subplots(ncols=2,figsize=(9.7, 6.27))
sns.boxplot(x="quality",y='residual sugar',data=wine, ax=axs[0])
sns.boxplot(x="categoria",y='residual sugar',data=wine, ax=axs[1])

In [ ]:
fig, axs = plt.subplots(ncols=2,figsize=(9.7, 6.27))
sns.boxplot(x="quality",y='chlorides',data=wine, ax=axs[0])
sns.boxplot(x="categoria",y='chlorides',data=wine, ax=axs[1])

In [ ]:
fig, axs = plt.subplots(ncols=2,figsize=(9.7, 6.27))
sns.boxplot(x="quality",y='free sulfur dioxide',data=wine, ax=axs[0])
sns.boxplot(x="categoria",y='free sulfur dioxide',data=wine, ax=axs[1])

In [ ]:
fig, axs = plt.subplots(ncols=2,figsize=(9.7, 6.27))
sns.boxplot(x="quality",y='total sulfur dioxide',data=wine, ax=axs[0])
sns.boxplot(x="categoria",y='total sulfur dioxide',data=wine, ax=axs[1])

In [ ]:
fig, axs = plt.subplots(ncols=2,figsize=(9.7, 6.27))
sns.boxplot(x="quality",y='density',data=wine, ax=axs[0])
sns.boxplot(x="categoria",y='density',data=wine, ax=axs[1])

In [ ]:
fig, axs = plt.subplots(ncols=2,figsize=(9.7, 6.27))
sns.boxplot(x="quality",y='pH',data=wine, ax=axs[0])
sns.boxplot(x="categoria",y='pH',data=wine, ax=axs[1])

In [ ]:
fig, axs = plt.subplots(ncols=2,figsize=(9.7, 6.27))
sns.boxplot(x="quality",y='sulphates',data=wine, ax=axs[0])
sns.boxplot(x="categoria",y='sulphates',data=wine, ax=axs[1])

In [ ]:
fig, axs = plt.subplots(ncols=2,figsize=(9.7, 6.27))
sns.boxplot(x="quality",y='alcohol',data=wine, ax=axs[0])
sns.boxplot(x="categoria",y='alcohol',data=wine, ax=axs[1])

### Correlação

As variáveis apresentam altas correlações. Olhando para o conjunto com todos os dados, entre as variáveis com os maiores valores, chamam a atenção 0.67 de correlação entre 'fixed acidity' e 'citric acid', 0.67 entre 'fixed acidity' e 'density', -0.68 entre 'fixed acidity' e 'ph', e 0.67 entre 'free sulphor dioxide' e 'total sulphor dioxide'.

Se filtrarmos pela categoria 'Bom' temos 0.7 de correlação entre 'fixed acidity' e 'citric acid', 0.7 entre 'fixed acidity' e 'density', -0.73 entre 'fixed acidity' e 'ph', e 0.67 entre 'free sulphor dioxide' e 'total sulphor dioxide'. Muito semelhantes às correlações do conjunto todo.

Para a categoria 'Ruim' as correlações parecem seguir valores semelhantes. Maiores para alguns pares e menores para outros, mas ainda assim semelhante ao conjunto todo.

In [ ]:
#Correlação
sns.pairplot(wine, hue = 'categoria', markers = '+')
plt.show()

In [ ]:
#Matriz de Correlação
plt.figure(figsize=(15,10))
sns.heatmap(wine.corr(), annot=True, fmt='.2f', linewidths=2)

In [ ]:
#Matriz de Correlação
plt.figure(figsize=(15,10))
sns.heatmap(wine[wine['categoria']=='Bom'].corr(), annot=True, fmt='.2f', linewidths=2)

In [ ]:
wine[wine['categoria']=='Bom'].corr()

In [ ]:
#Matriz de Correlação
plt.figure(figsize=(15,10))
sns.heatmap(wine[wine['categoria']=='Ruim'].corr(), annot=True, fmt='.2f', linewidths=2)

In [ ]:
wine[wine['categoria']=='Ruim'].corr()

## Classificadores

Foram utilizados três classificadores nesse trabalho: Análise de discriminante linear (LDA), Análise de discriminante quadrática (QDA) e Naive Bayes Gaussiano. Dividimos as amostras em 70% para treino e 30% para teste.

In [ ]:
wineCopy = wine.copy()

In [ ]:
X = wineCopy.drop(columns = ['categoria','quality'])
y = [1 if value == 'Bom' else 0 for value in wineCopy['categoria']]
results = []

In [ ]:
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify = y, test_size = test_size)

### Análise de discriminante linear

In [ ]:
#lda
LDA = LinearDiscriminantAnalysis()
LDA.fit(X_train, y_train)
y_pred_lda = LDA.predict(X_test)

CM = confusion_matrix(y_test, y_pred_lda)
acc = accuracy_score(y_test, y_pred_lda)
#score = LDA.score(X_test, y_test)
score = cross_val_score(LDA, X, y, cv=5) # cross-validation
results.append(acc)

print("Score : ", score.mean())
print("Linear Discriminant Analysis Accuracy: ", acc)

plot_confusion_matrix(LDA, X_test, y_test, cmap= "Blues")  
plt.show()

In [ ]:
metrics.plot_roc_curve(LDA, X_test, y_test)

In [ ]:
target_names = ['Ruim', 'Bom']
print(classification_report(y_test, y_pred_lda, target_names=target_names))

In [ ]:
cross_val_score(LDA, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.roc_auc_score, greater_is_better = True, needs_proba = True))

In [ ]:
cross_val_score(LDA, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False))

### Análise de discriminante quadrático

In [ ]:
#qda
QDA = QuadraticDiscriminantAnalysis()
QDA.fit(X_train, y_train)
y_pred_qda = QDA.predict(X_test)
CM = confusion_matrix(y_test, y_pred_qda)

acc = accuracy_score(y_test, y_pred_qda)
#score = QDA.score(X_test, y_test)
score = cross_val_score(QDA, X, y, cv=5) # cross-validation
results.append(acc)

print("Score : ", score.mean())
print("Quadratic Discriminant Analysis Accuracy: ", acc)

plot_confusion_matrix(QDA, X_test, y_test, cmap= "Blues")  
plt.show()

In [ ]:
metrics.plot_roc_curve(QDA, X_test, y_test)

In [ ]:
target_names = ['Ruim', 'Bom']
print(classification_report(y_test, y_pred_qda, target_names=target_names))

In [ ]:
cross_val_score(QDA, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.roc_auc_score, greater_is_better = True, needs_proba = True))

In [ ]:
cross_val_score(QDA, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False))

### Naive Bayes Gaussiano

In [ ]:
#Gaussian Naive Bayes
NB = GaussianNB()
NB.fit(X_train, y_train)
y_pred_nb = NB.predict(X_test)
CM = confusion_matrix(y_test, y_pred_nb)

acc = accuracy_score(y_test, y_pred_nb)
#score = NB.score(X_test, y_test)
score = cross_val_score(NB, X, y, cv=5) # cross-validation
results.append(acc)

print("Score : ", score.mean())
print("Gaussian Naive Bayes Accuracy: ", acc)

plot_confusion_matrix(NB, X_test, y_test, cmap= "Blues")  
plt.show()

In [ ]:
metrics.plot_roc_curve(NB, X_test, y_test)

In [ ]:
target_names = ['Ruim', 'Bom']
print(classification_report(y_test, y_pred_nb, target_names=target_names))

In [ ]:
cross_val_score(NB, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.roc_auc_score, greater_is_better = True, needs_proba = True))

In [ ]:
cross_val_score(NB, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False))

## Comparando

Foi utilizada validação cruzada para 5 iterações para cada um dos modelos. O modelo que apresentou a melhor acurácia foi o LDA com 0.87, contra 0.83 do QDA e 0.84 do Naive Bayes Gaussiano. O plot de comparação entre a curva ROC de cada modelo confirma essa informação, onde a maior área entre a curva também é do LDA com AUC igual a 0.89. Apesar da alta acurácia , ao observarmos a matriz de confusão o modelo LDA apresenta a maior taxa de falsos positivos.

In [ ]:
print("LDA cross-validation score: ", cross_val_score(LDA, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.roc_auc_score, greater_is_better = True, needs_proba = True)).mean())

print("QDA cross-validation score: ",cross_val_score(QDA, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.roc_auc_score, greater_is_better = True, needs_proba = True)).mean())

print("Naive Bayes cross-validation score: ",cross_val_score(NB, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.roc_auc_score, greater_is_better = True, needs_proba = True)).mean())

In [ ]:
print(cross_val_score(NB, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False)).mean())

print(cross_val_score(LDA, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False)).mean())

print(cross_val_score(QDA, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False)).mean())

In [ ]:
plt.figure(figsize=(10,8))

fig1 = metrics.plot_roc_curve(LDA, X_test, y_test)

fig2 = metrics.plot_roc_curve(QDA, X_test, y_test,ax=fig1.ax_)
fig2.figure_.suptitle("ROC curve comparison")

fig3 = metrics.plot_roc_curve(NB, X_test, y_test,ax=fig1.ax_)
fig3.figure_.suptitle("ROC curve comparison")

plt.legend(loc=0)

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred_lda))
print(metrics.accuracy_score(y_test, y_pred_lda))
print(metrics.f1_score(y_test, y_pred_lda))

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred_qda))
print(metrics.accuracy_score(y_test, y_pred_qda))
print(metrics.f1_score(y_test, y_pred_qda))

In [ ]:
print(metrics.confusion_matrix(y_test, y_pred_nb))
print(metrics.accuracy_score(y_test, y_pred_nb))
print(metrics.f1_score(y_test, y_pred_nb))

## Conclusão

A análise exploratória dos dados de vinhos mostrou um viéis grande quando selecionamos a nota de corte para bons vinhos como maiores ou iguais a 7, com a maior parte das observações sendo consideradas ruins. Além disso, o número extremamente grande de outliers parece ter afetado a precisão da modelagem. Seria interessante remover essas observações e testar os modelos novamente. A alta correlação entre as variáveis pode ter tido um efeito negativo nas previsões também.

Foram utilizados os três classificadores apresentados em aula: LDA, QDA e Naive Bayes. Estes modelos não apresentaram resultados muito diferentes em relação a acurácia, se diferenciando mais em relação a captura de casos verdadeiros positivos e verdadeiros negativos. O LDA apresentou uma vantagem marginal em relação aos outros em termos de acurácia e AUC, logo foi selecionado como o melhor classificador para esse trabalho. 

O modelo selecionado como o melhor é muito bom para a classificação de verdadeiros negativos, ou seja, detectar os casos em que o vinho está entre os níveis de qualidade 3 e 6 inclusive. No entanto, a taxa de falsos positivos é muito alta. Se a demanda fosse pela detecção dos vinhos bons com alta precisão, deveríamos investigar um outro modelo mais eficiente.

## Apêndice: Alguns outros modelos

Aqui foram testados alguns outros modelos que não foram vistos em aula, apenas como teste. Tentei implementar o SVM visto na aula 5, mas não deu muito certo.

### Logistic Regression

In [ ]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='liblinear',multi_class='ovr')
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
cm = confusion_matrix(y_test, y_pred_lr)

acc = accuracy_score(y_test, y_pred_lr)
#score = NB.score(X_test, y_test)
score = cross_val_score(lr, X, y, cv=5) # cross-validation
results.append(acc)

print("Score : ", score.mean())
print("Logistic Regression Accuracy: ", acc)

plot_confusion_matrix(lr, X_test, y_test, cmap= "Blues")  
plt.show()

In [ ]:
metrics.plot_roc_curve(lr, X_test, y_test)

In [ ]:
target_names = ['Ruim', 'Bom']
print(classification_report(y_test, y_pred_lr, target_names=target_names))

In [ ]:
cross_val_score(lr, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.roc_auc_score, greater_is_better = True, needs_proba = True))

In [ ]:
cross_val_score(lr, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False))


In [ ]:
cross_val_score(lr, X, y,cv=5)

### Random Forest

In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=40)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred_rf)

acc = accuracy_score(y_test, y_pred_rf)
#score = NB.score(X_test, y_test)
score = cross_val_score(rf, X, y, cv=5) # cross-validation
results.append(acc)

print("Score : ", score.mean())
print("Random Forest Accuracy: ", acc)

plot_confusion_matrix(rf, X_test, y_test, cmap= "Blues")  
plt.show()

In [ ]:
metrics.plot_roc_curve(rf, X_test, y_test)

In [ ]:
target_names = ['Ruim', 'Bom']
print(classification_report(y_test, y_pred_rf, target_names=target_names))

In [ ]:
cross_val_score(rf, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.roc_auc_score, greater_is_better = True, needs_proba = True))

In [ ]:
cross_val_score(rf, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False))

In [ ]:
cross_val_score(rf, X, y,cv=5)

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred_dt = rf.predict(X_test)
cm = confusion_matrix(y_test,y_pred_dt)

acc = accuracy_score(y_test, y_pred_dt)
#score = NB.score(X_test, y_test)
score = cross_val_score(dt, X, y, cv=5) # cross-validation
results.append(acc)

print("Score : ", score.mean())
print("Random Forest Accuracy: ", acc)

plot_confusion_matrix(rf, X_test, y_test, cmap= "Blues")  
plt.show()

In [ ]:
metrics.plot_roc_curve(dt, X_test, y_test)

In [ ]:
target_names = ['Ruim', 'Bom']
print(classification_report(y_test, y_pred_dt, target_names=target_names))

### Comparando com outros modelos

In [ ]:
print("LDA cross-validation score: ", cross_val_score(LDA, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.roc_auc_score, greater_is_better = True, needs_proba = True)).mean())

print("QDA cross-validation score: ",cross_val_score(QDA, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.roc_auc_score, greater_is_better = True, needs_proba = True)).mean())

print("Naive Bayes cross-validation score: ",cross_val_score(NB, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.roc_auc_score, greater_is_better = True, needs_proba = True)).mean())

print("Logistic regression cross-validation score: ",cross_val_score(lr, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.roc_auc_score, greater_is_better = True, needs_proba = True)).mean())

print("Random forest cross-validation score: ",cross_val_score(rf, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.roc_auc_score, greater_is_better = True, needs_proba = True)).mean())

print("Decision tree cross-validation score: ",cross_val_score(dt, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.roc_auc_score, greater_is_better = True, needs_proba = True)).mean())

In [ ]:
plt.figure(figsize=(10,8))

fig1 = metrics.plot_roc_curve(LDA, X_test, y_test)

fig2 = metrics.plot_roc_curve(QDA, X_test, y_test,ax=fig1.ax_)
fig2.figure_.suptitle("ROC curve comparison")

fig3 = metrics.plot_roc_curve(NB, X_test, y_test,ax=fig1.ax_)
fig3.figure_.suptitle("ROC curve comparison")

fig4 = metrics.plot_roc_curve(lr, X_test, y_test,ax=fig1.ax_)
fig4.figure_.suptitle("ROC curve comparison")

fig5 = metrics.plot_roc_curve(rf, X_test, y_test,ax=fig1.ax_)
fig5.figure_.suptitle("ROC curve comparison")

fig6 = metrics.plot_roc_curve(dt, X_test, y_test,ax=fig1.ax_)
fig6.figure_.suptitle("ROC curve comparison")

plt.legend(loc=0)

In [ ]:
print("LDA cross-validation score: ", cross_val_score(LDA, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False)).mean())
      
print("QDA cross-validation score: ",cross_val_score(QDA, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False)).mean())

print("Naive Bayes cross-validation score: ",cross_val_score(NB, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False)).mean())
      
print("Logistic regression cross-validation score: ",cross_val_score(lr, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False)).mean())
      
print("Random forest cross-validation score: ",cross_val_score(rf, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False)).mean())
      
print("Decision tree cross-validation score: ",cross_val_score(dt, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False)).mean())
    